In [ ]:
from PIL import Image, ImageDraw
import numpy as np
import os

def resize_and_crop_to_circle(image, diameter=300):
    """Resizes an image to fit a circle with the specified diameter,
    upscaling or downscaling if necessary, and then crops it to a circle
    based on the transparency mask. Crops to the mask first, before resizing.
    """

    img = Image.open(image).convert("RGBA")

    # Create transparency mask and get nonzero pixel indices
    mask = img.getchannel("A")  # Extract alpha channel as mask
    nonzero_indices = np.nonzero(mask)  # Get indices of non-transparent pixels

    # Find bounding box of non-transparent pixels
    y_min, y_max = nonzero_indices[0].min(), nonzero_indices[0].max()
    x_min, x_max = nonzero_indices[1].min(), nonzero_indices[1].max()

    # Crop to the bounding box
    cropped_img = img.crop((x_min, y_min, x_max, y_max))

    resize_ratio = diameter / max(cropped_img.width, cropped_img.height)

    # Resize based on the calculated ratio using LANCZOS filter
    resized_img = cropped_img.resize((int(cropped_img.width * resize_ratio), int(cropped_img.height * resize_ratio)), Image.LANCZOS)

    # Ensure bounding box fits within circle diameter (after resizing)
    box_width = resized_img.width
    box_height = resized_img.height
    if box_width > diameter or box_height > diameter:
        crop_width = min(box_width, diameter)
        crop_height = min(box_height, diameter)
        x_offset = (box_width - crop_width) // 2
        y_offset = (box_height - crop_height) // 2
        box_width = crop_width
        box_height = crop_height
    else:
        x_offset = 0
        y_offset = 0

    # Crop to the circle within the bounding box (after resizing)
    diameter = min(box_width, box_height)  # Adjust diameter for smaller bounding box
    mask = Image.new("L", resized_img.size, 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((0, 0, diameter, diameter), fill=255)
    return resized_img.crop((0, 0, diameter, diameter)).convert("RGBA")


def create_grid(images, rows, cols, spacing=0, diameter=300):
  """Arranges images in a grid with optional spacing."""
  cell_width = diameter + spacing
  cell_height = diameter + spacing
  grid_width = cell_width * cols
  grid_height = cell_height * rows
  grid = Image.new("RGBA", (grid_width, grid_height), (0, 0, 0, 0))

  for i, image in enumerate(images):
    x = (i % cols) * cell_width
    y = (i // cols) * cell_height
    # Directly paste the pre-processed image from resized_images list
    grid.paste(image, (x, y))

  return grid

In [ ]:
source_dir = "Source_Images"
filename = "StickerSheet.png"
quality = 95
# Adjust rows and cols based on desired sheet size and image diameter 
rows = 9  # Adjust as needed
cols = 6  # Adjust as needed
spacing = 30  # Adjust as needed

images = [os.path.join(source_dir, f) for f in os.listdir(source_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
resized_images = [resize_and_crop_to_circle(img) for img in images]
grid = create_grid(resized_images, rows, cols, spacing)
grid.save(filename, quality=quality)

In [ ]:
display(Image.open(filename))